In [ ]:
!pip install gradio transformers torch psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.9 MB/s eta 0:00:00


In [ ]:
import re
import gradio as gr
from datetime import datetime
import logging
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline


In [ ]:
# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)


In [ ]:
# Helper Functions (assumed from prior code)
def clean_numeric_input(value):
    try:
        cleaned = re.sub(r'[^\d.]', '', str(value))
        return float(cleaned)
    except:
        return None

def build_prompt(history, user_input):
    convo = ""
    if history:
        for msg in history[-5:]:
            convo += f"{msg['role'].capitalize()}: {msg['content']}\n"

    prompt = f"""
You are a friendly and detailed personal finance coach. Provide clear, actionable, and specific advice tailored to the user's question. Avoid repetitive phrases, redundant sentences, or generic responses. Ensure the response is concise, practical, and directly addresses the query. Do not include the word 'Coach' or repeat the user's input verbatim. If the user asks about saving for goals, include specific examples, reference goal-setting commands, and suggest financial tools. Use the following examples as a guide for tone and structure:

Example 1:
User: How can I save money on a tight budget?
Response: Track every expense to identify savings opportunities. Follow the 50/30/20 rule: 50% for needs, 30% for wants, 20% for savings. Cancel unused subscriptions and reduce dining out.

Example 2:
User: I want to buy a car in 2 years, how to save?
Response: Estimate the car’s cost (e.g., $20,000) and divide by 24 to set a monthly savings target of about $833. Use 'add goal car 20000 24' to track it. Open a high-yield savings account to earn interest and automate monthly transfers. Cut non-essential expenses or consider a side hustle.

Example 3:
User: How can I save for goals?
Response: Define each goal, like a vacation or home, and estimate its cost. Use 'add goal vacation 1000 12' to set a target and timeframe. Divide the cost by the months to find your savings target. Save in a high-yield savings account and automate transfers to stay consistent.

Conversation so far:
{convo}
User: {user_input}
Response:
"""
    return prompt.strip()




In [ ]:
def deduplicate_response(text):
    # Extract text after "Response:" and remove prompt artifacts
    text = text.split("Response:")[-1].strip() if "Response:" in text else text.strip()

    # Split into sentences, handling T5 output quirks
    lines = [line.strip() for line in re.split(r'[.!?]\s+', text) if line.strip()]
    seen = set()
    filtered = []
    for line in lines:
        line_clean = line.lower().strip()
        if line_clean and line_clean not in seen:
            filtered.append(line)
            seen.add(line_clean)

    # Join sentences with proper punctuation
    result = '. '.join(filtered)
    if result and not result.endswith('.'):
        result += '.'
    return result if result else "Please clarify your request."

In [ ]:
# FinanceAgent class (partial, with required methods)
class FinanceAgent:
    def __init__(self):
        self.history = []
        self.goals = {}
        self.budget = {"income": 0.0, "expenses": 0.0}

    def reset(self):
        self.history = []
        self.goals = {}
        self.budget = {"income": 0.0, "expenses": 0.0}
        return "Session reset. Welcome to your Personal Finance Coach! Try 'add goal vacation 1000 12' or ask 'How can I save for goals?'"

    def add_goal(self, name, amount, timeframe=None):
        amount_val = clean_numeric_input(amount)
        if amount_val is None or not name.strip():
            return "Invalid input. Use 'add goal name amount [months]' (e.g., add goal vacation 1000 12)."
        goal_data = {"amount": amount_val, "date_added": datetime.now().strftime("%Y-%m-%d")}
        if timeframe:
            tf = clean_numeric_input(timeframe)
            if tf and tf > 0:
                goal_data["monthly_saving"] = amount_val / tf
        self.goals[name.lower()] = goal_data
        msg = f"Goal '{name}' added with target ${amount_val:.2f}."
        if "monthly_saving" in goal_data:
            msg += f" Save ${goal_data['monthly_saving']:.2f}/month for {tf} months."
        return msg

    def list_goals(self):
        if not self.goals:
            return "No goals set. Try 'add goal vacation 1000 12' to start."
        s = "Your goals:\n"
        for name, data in self.goals.items():
            s += f"- {name}: ${data['amount']:.2f} (Added {data['date_added']})"
            if "monthly_saving" in data:
                s += f", Monthly: ${data['monthly_saving']:.2f}"
            s += "\n"
        return s.strip()

    def estimate_savings(self, income, expenses):
        income_val = clean_numeric_input(income)
        expenses_val = clean_numeric_input(expenses)
        if income_val is None or expenses_val is None:
            return "Enter valid numbers (e.g., estimate savings 3000 2500)."
        savings = income_val - expenses_val
        self.budget["income"] = income_val
        self.budget["expenses"] = expenses_val
        return f"Savings: ${savings:.2f}/month" if savings > 0 else "Expenses exceed income."

    def get_budget_summary(self):
        if self.budget["income"] == 0 and self.budget["expenses"] == 0:
            return "No budget data yet. Try 'estimate savings 3000 2500' to set income and expenses."
        savings = self.budget["income"] - self.budget["expenses"]
        return f"Income: ${self.budget['income']:.2f}\nExpenses: ${self.budget['expenses']:.2f}\nSavings: ${savings:.2f}/month"

    def respond(self, user_input):
        if not model:
            return "Model failed to load. Check logs and ensure dependencies are installed."
        self.history.append({"role": "user", "content": user_input})
        lower = user_input.lower().strip()
        logger.debug(f"Processed input: {lower}")

        if lower.startswith("add goal"):
            match = re.match(r"add goal\s+(.+?)\s+(\d*\.?\d*)\s*(\d*\.?\d*)?", lower)
            if match:
                name, amount, timeframe = match.groups()
                msg = self.add_goal(name, amount, timeframe or None)
                self.history.append({"role": "coach", "content": msg})
                return msg
            return "Invalid format. Use 'add goal name amount [months]'."

        if lower.startswith("list goals"):
            msg = self.list_goals()
            self.history.append({"role": "coach", "content": msg})
            return msg

        if lower.startswith("estimate savings"):
            match = re.match(r"estimate savings\s+(\d*\.?\d*)\s+(\d*\.?\d*)", lower)
            if match:
                income, expenses = match.groups()
                msg = self.estimate_savings(income, expenses)
                self.history.append({"role": "coach", "content": msg})
                return msg
            return "Invalid format. Use 'estimate savings income expenses'."

        if "save for goals" in lower:
            msg = "Define each goal, like a vacation or home, and estimate its cost. Use 'add goal vacation 1000 12' to set a target and timeframe. Divide the cost by the months to find your savings target. Save in a high-yield savings account and automate transfers."
            if self.goals:
                msg += " Your current goals include:\n" + self.list_goals()
            self.history.append({"role": "coach", "content": msg})
            return msg

        if "buy a car" in lower and "save" in lower:
            msg = "Estimate the car’s cost (e.g., $20,000) and divide by 24 to set a monthly savings target of about $833. Use 'add goal car 20000 24' to track it. Open a high-yield savings account to earn interest and automate monthly transfers. Cut non-essential expenses or consider a side hustle."
            if self.goals:
                msg += " Your current goals include:\n" + self.list_goals()
            self.history.append({"role": "coach", "content": msg})
            return msg

        prompt = build_prompt(self.history, user_input)
        logger.debug(f"Prompt: {prompt}")
        try:
            output = model(
                prompt,
                max_length=300,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                num_beams=5,
                early_stopping=True
            )[0]['generated_text'].strip()
            logger.debug(f"Raw model output: {output}")

            if "Response:" in output:
                output = output.split("Response:")[-1].strip()
            else:
                output = output.strip()

            output = deduplicate_response(output)
            if not output or output.lower() in ["coach", "response"]:
                output = "Please clarify your request."
        except Exception as e:
            logger.error(f"Model error: {e}")
            output = "Error generating response. Check logs."

        self.history.append({"role": "coach", "content": output})
        return output

In [ ]:
# Gradio UI logic
def user_interaction(user_input, chat_history, agent_state):
    logger.debug(f"agent_state type: {type(agent_state)}, value: {agent_state}")
    agent = agent_state if isinstance(agent_state, FinanceAgent) else FinanceAgent()
    output = agent.respond(user_input)
    chat_history = chat_history or []
    chat_history.append({"role": "user", "content": user_input})
    chat_history.append({"role": "assistant", "content": output})
    return "", chat_history, agent, agent.list_goals(), agent.get_budget_summary()

def reset_chat(chat_history, agent_state):
    logger.debug(f"agent_state type: {type(agent_state)}, value: {agent_state}")
    agent = agent_state if isinstance(agent_state, FinanceAgent) else FinanceAgent()
    msg = agent.reset()
    chat_history = [{"role": "assistant", "content": msg}]
    return chat_history, agent, agent.list_goals(), agent.get_budget_summary()

def example_prompt(prompt, chat_history, agent_state):
    return user_interaction(prompt, chat_history, agent_state)

# Model initialization for google/flan-t5-base
try:
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
    model_base = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
    model = pipeline("text2text-generation", model=model_base, tokenizer=tokenizer)
except Exception as e:
    logger.error(f"Model loading failed: {e}")
    model = None

Device set to use cpu


In [ ]:

# Launch UI
try:
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 💰 Personal Finance Assistant
            Plan your financial future! Add goals, estimate savings, or ask for advice.
            **Examples**: 'add goal vacation 1000 12', 'list goals', 'estimate savings 3000 2500', 'How can I save for goals?'
            """
        )

        with gr.Row():
            # Main chat area
            with gr.Column(scale=3):
                user_input = gr.Textbox(
                    placeholder="Type your question or command (e.g., 'add goal vacation 1000 12')",
                    label="Your Input"
                )
                with gr.Row():
                    send_btn = gr.Button("Send", variant="primary")
                    reset_btn = gr.Button("Reset Chat", variant="secondary")
                chat = gr.Chatbot(type="messages", label="Chat with Your Finance Coach", height=400)

            # Sidebar for goals and budget
            with gr.Column(scale=1):
                gr.Markdown("### Your Financial Snapshot")
                goals_display = gr.Textbox(label="Saved Goals", lines=5, interactive=False)
                budget_display = gr.Textbox(label="Budget Summary", lines=3, interactive=False)
                gr.Markdown("### Try These Prompts")
                with gr.Column():
                    example_btn1 = gr.Button("Add a Goal")
                    example_btn2 = gr.Button("List Goals")
                    example_btn3 = gr.Button("Estimate Savings")
                    example_btn4 = gr.Button("Save for Goals")
                    example_btn5 = gr.Button("Stay Motivated")

        # Help section
        with gr.Accordion("How to Use", open=False):
            gr.Markdown("""
            - **Add a Goal**: Use 'add goal name amount [months]' (e.g., 'add goal vacation 1000 12').
            - **List Goals**: Type 'list goals' to see your saved goals.
            - **Estimate Savings**: Use 'estimate savings income expenses' (e.g., 'estimate savings 3000 2500').
            - **Ask Questions**: Try 'How can I save for goals?' or 'How do I stay motivated to stick to my budget?'.
            - **Reset**: Click 'Reset Chat' to start over.
            """)

        agent_state = gr.State(FinanceAgent())

        # Event handlers
        send_btn.click(
            fn=user_interaction,
            inputs=[user_input, chat, agent_state],
            outputs=[user_input, chat, agent_state, goals_display, budget_display]
        )
        reset_btn.click(
            fn=reset_chat,
            inputs=[chat, agent_state],
            outputs=[chat, agent_state, goals_display, budget_display]
        )
        example_btn1.click(
            fn=example_prompt,
            inputs=[gr.State(value="add goal vacation 1000 12"), chat, agent_state],
            outputs=[user_input, chat, agent_state, goals_display, budget_display]
        )
        example_btn2.click(
            fn=example_prompt,
            inputs=[gr.State(value="list goals"), chat, agent_state],
            outputs=[user_input, chat, agent_state, goals_display, budget_display]
        )
        example_btn3.click(
            fn=example_prompt,
            inputs=[gr.State(value="estimate savings 3000 2500"), chat, agent_state],
            outputs=[user_input, chat, agent_state, goals_display, budget_display]
        )
        example_btn4.click(
            fn=example_prompt,
            inputs=[gr.State(value="How can I save for goals?"), chat, agent_state],
            outputs=[user_input, chat, agent_state, goals_display, budget_display]
        )
        example_btn5.click(
            fn=example_prompt,
            inputs=[gr.State(value="How do I stay motivated to stick to my budget?"), chat, agent_state],
            outputs=[user_input, chat, agent_state, goals_display, budget_display]
        )

    demo.launch(debug=True, share=True)
except Exception as e:
    logger.error(f"Gradio launch failed: {e}")
    print(f"Error launching Gradio: {e}. Restart runtime and reinstall dependencies if needed.")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://90073500b0db38f9e7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
